In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [10]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
         available = soup.find("div", attrs={'id':'availability'})
         available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [11]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptps&crid=2AOBGX0LLRANS&sprefix=laptps%2Caps%2C528&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

C:\Users\JESIKA\AppData\Local\Temp\ipykernel_4380\1231261326.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [13]:
amazon_df

,title,price,rating,reviews,availability
0,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",,4.3 out of 5 stars,"1,755 ratings",Only 8 left in stock - order soon.
1,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,,4.4 out of 5 stars,"39,133 ratings",In Stock
2,"HP Stream 14"" HD BrightView Laptop, Intel Pent...",,4.3 out of 5 stars,14 ratings,Only 7 left in stock - order soon.
3,"MSI Katana 15 15.6"" 144Hz FHD Gaming Laptop: 1...",,4.5 out of 5 stars,360 ratings,Not Available
4,MSI Stealth 18 AI Studio 18” 240Hz QHD+ Gaming...,,4.5 out of 5 stars,"1,050 ratings",In Stock
5,"acer Nitro V Gaming Laptop, 16"" FHD 165Hz, AMD...",,4.5 out of 5 stars,210 ratings,Not Available
6,"Lenovo V15 Laptop, 15.6"" FHD Display, AMD Ryze...",,4.4 out of 5 stars,433 ratings,Not Available
7,"Lenovo IdeaPad 1 Student Laptop, Intel Dual Co...",,4.4 out of 5 stars,718 ratings,Not Available
8,"HP Newest 14"" Ultral Light Laptop for Students...",,4.3 out of 5 stars,"1,983 ratings",In Stock
9,"Laptop, 17.3 Inch Laptops Computer, 4GB DDR3 1...",,4.3 out of 5 stars,52 ratings,In Stock


In [14]:
print(amazon_df['title'])

0     HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...
1     Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...
2     HP Stream 14" HD BrightView Laptop, Intel Pent...
3     MSI Katana 15 15.6" 144Hz FHD Gaming Laptop: 1...
4     MSI Stealth 18 AI Studio 18” 240Hz QHD+ Gaming...
5     acer Nitro V Gaming Laptop, 16" FHD 165Hz, AMD...
6     Lenovo V15 Laptop, 15.6" FHD Display, AMD Ryze...
7     Lenovo IdeaPad 1 Student Laptop, Intel Dual Co...
8     HP Newest 14" Ultral Light Laptop for Students...
9     Laptop, 17.3 Inch Laptops Computer, 4GB DDR3 1...
10    ACEMAGIC 2024 News Gaming Laptop,AMD Ryzen 7 5...
11    17.3 Inch Laptops Computer, 8GB DDR4 512GB SSD...
12    ACEMAGIC Laptop, Windows 11 Laptop Computer,16...
13    HP Stream 14" HD BrightView Laptop, Intel Cele...
14    HP 15.6" Portable Laptop (Include 1 Year Micro...
15    HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...
16    HP Elitebook 650 G10 15.6" FHD Business Laptop...
17    Laptop, 17.3 Inch Laptops Computer, 4GB DD

In [15]:
print(amazon_df['rating'])

0     4.3 out of 5 stars
1     4.4 out of 5 stars
2     4.3 out of 5 stars
3     4.5 out of 5 stars
4     4.5 out of 5 stars
5     4.5 out of 5 stars
6     4.4 out of 5 stars
7     4.4 out of 5 stars
8     4.3 out of 5 stars
9     4.3 out of 5 stars
10    4.5 out of 5 stars
11    4.7 out of 5 stars
12    4.5 out of 5 stars
13    4.3 out of 5 stars
14    4.6 out of 5 stars
15    4.3 out of 5 stars
16    4.4 out of 5 stars
17    4.7 out of 5 stars
18    4.4 out of 5 stars
19    4.4 out of 5 stars
20    4.3 out of 5 stars
21    4.3 out of 5 stars
22    4.5 out of 5 stars
23    4.3 out of 5 stars
24    4.4 out of 5 stars
25    4.4 out of 5 stars
Name: rating, dtype: object
